# Import

In [ ]:

import tensorflow as tf
tf.keras.backend.clear_session()

from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.initializers import HeNormal, GlorotUniform
from tensorflow.keras.callbacks import TensorBoard
from tensorboard.plugins import projector
from tensorflow.keras.applications import MobileNet, MobileNetV3Small,ResNet50V2, Xception, ConvNeXtSmall, EfficientNetB3
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adagrad
from tensorflow.keras import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import * 
import tensorflow_datasets as tfds

from itertools import combinations, product, islice
import os
import glob
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import cv2
import albumentations as A

from contrastive_loss import contrastiveLoss
from dataLoader import Loader
from circle_loss import circleLoss
from siamese_network import siameseNetwork
from distance import Distance
from accuracy import Accuracy
from sim_distance import similarityDistances
from activation_callback import actionCallback

import multiprocessing
print("Number of logical CPU cores:", multiprocessing.cpu_count())

2025-04-13 16:49:15.451936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-13 16:49:15.574481: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 16:49:16.280475: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cdiop/.conda/envs/perso/lib:/lib
2025-04-13 16:49:16.280532: W tensorflow/compile

Number of logical CPU cores: 48


# Model

In [ ]:
def get_feature_extractor(input_shape, param_list, l2reg=None, pool_size=None):
    
    model = Sequential([
        tf.keras.layers.Input(shape=input_shape),
    ])
    
    for i, param in enumerate(param_list):
        
        model.add(tf.keras.layers.Conv2D(
            filters=param[0], kernel_size=param[1], padding=param[2], strides=param[3], kernel_initializer='he_normal', name = f"conv{i}",
            kernel_regularizer=regularizers.l2(l2reg) if l2reg else None 
        ))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.MaxPool2D((pool_size,pool_size), padding="same")) if pool_size else None
    
    return model


In [ ]:
def get_feature_extractor_2(l2reg=None):

    feature_extractor = Sequential([
        tf.keras.layers.Input(shape=(128,128,3)),
        
        Conv2D(filters = 16, kernel_size = 5, padding="same", strides=1,  kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        Conv2D(filters = 16, kernel_size = 1, padding="valid", strides=1,  kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        
        MaxPool2D(3, strides=2),
        
        Conv2D(filters = 64, kernel_size = 3, padding="same", strides=1,  kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        Conv2D(filters = 64, kernel_size = 1, padding="valid", strides=1,  kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(3, strides=2),
        
        Conv2D(filters = 96, kernel_size = 3, padding="valid", strides=1,  kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        Conv2D(filters = 96, kernel_size = 1, padding="valid", strides=1,  kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(3, strides=2),
        
        Conv2D(filters = 128, kernel_size = 3, padding="valid", strides=1,  kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        Conv2D(filters = 128, kernel_size = 1, padding="valid", strides=1,  kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(3, strides=2),
        
    ])

    return feature_extractor

In [ ]:
def get_feature_extractor_3(l2reg):

    feature_extractor = Sequential([
        tf.keras.layers.Input(shape=(128,128,3)),
        
        Conv2D(filters = 64, kernel_size = 5, padding="same", strides=1,kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        Conv2D(filters = 64, kernel_size = 1, padding="valid", strides=1, kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        
        MaxPool2D(2),
        
        Conv2D(filters = 96, kernel_size = 3, padding="same", strides=1, kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        Conv2D(filters = 96, kernel_size = 1, padding="valid", strides=1, kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(2),
        
        Conv2D(filters = 128, kernel_size = 3, padding="same", strides=1, kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        Conv2D(filters = 128, kernel_size = 1, padding="valid", strides=1, kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(2),
        
        Conv2D(filters = 256, kernel_size = 3, padding="same", strides=1, kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(2),
        Conv2D(filters = 256, kernel_size = 1, padding="valid", strides=1, kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(2),
        
        Conv2D(filters = 512, kernel_size = 3, padding="valid", strides=1,kernel_initializer='he_normal',  kernel_regularizer=regularizers.l2(l2reg) if l2reg else None),
        BatchNormalization(),
        LeakyReLU(),
        MaxPool2D(2),
        
    
    ])

    return feature_extractor

# f = get_feature_extractor_3()
# f.summary()